In [1]:
import tensorflow as tf
import keras

Using TensorFlow backend.


## 6.1.1 단어와 문자의 원-핫 인코딩

In [2]:
# 케라스를 사용한 단어 수준 one-hot encoding
from keras.preprocessing.text import Tokenizer

samples = ["The cat sat on the mat.", "The dog ate my homework."]

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples) # 단어 인덱스 구축

sequences = tokenizer.texts_to_sequences(samples) # 문자열을 정수 인덱스 리스트로 변환

one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))
tokenizer.index_word

9개의 고유한 토큰을 찾았습니다.


{1: 'the',
 2: 'cat',
 3: 'sat',
 4: 'on',
 5: 'mat',
 6: 'dog',
 7: 'ate',
 8: 'my',
 9: 'homework'}

In [3]:
# one-hot hashing : 어휘 사전에 고유 토큰 수가 너무 커서 모두 다루기 어려울 때 사용
# 각 단어에 명시적으로 인덱스 할당하는 것이 아니라 단어를 해싱하여 고정된 크기의 벡터로 변환
# 메모리 절약하고, 온라인 방식으로 데이터 인코딩할 수 있지만, 해시 충돌 발생 가능성 존재
import numpy as np
dimensionality = 1000 # 단어를 크기가 1000인 벡터로 저장. 1000개 이상의 단어가 존재할 경우 해싱 충돌이 늘어남
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) % dimensionality # 단어를 해싱하여 0~1000 사이의 랜덤한 정수 인덱스로 변환
        results[i,j,index] = 1.

## 6.1.2 단어 임베딩 사용하기

- 원-핫 인코딩 : sparse / 고차원(어휘 사전에 있는 단어의 수와 동일 차원)
- 단어 임베딩 : 저차원의 실수형 밀집 벡터
    - 원-핫 인코딩(수동 인코딩)으로 얻은 단어 벡터와 달리 데이터로부터 학습
    - 보통 256차원, 512차원을 사용하며 큰 어휘 사전의 경우 1024차원의 단어 임베딩 사용
      (원-핫 인코딩의 경우 2만개 토큰의 어휘 사전일 경우 2만 차원 혹은 그 이상의 벡터가 만들어짐)

#### Empedding 층을 사용하여 단어 임베딩 학습하기

In [4]:
# Embedding 층의 객체 생성하기
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64) # 파라미터: 토큰의 개수 / 임베딩 차원

- Embedding층 입력: (samples, sequence_length)의 2D 정수 텐서
- Embedding층 출력: (samples, sequence_length, embedding_dimensionality) 3D 실수형 텐서

In [2]:
# IMDB 영화 리뷰 감성 예측 문제로 실습
# 가장 빈도 높은 1만개 단어 추출 후 20개 단어 이후는 버림
# 1만개 단어에 대해 8차원의 임베딩을 학습하여 정수 시퀀스 입력(2D 정수 텐서)을
# 임베딩 시퀀스(3D 실수형 텐서)로 변환
# 그 후, 이 텐서를 2D로 펼쳐서 분류를 위한 Dense층 훈련

In [5]:
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000 #특성으로 사용할 단어 수
maxlen = 20 # 사용할 텍스트 길이

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# 정수 리스트 로드

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환

In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length=maxlen)) #(samples, maxlen, 8) 출력

model.add(Flatten()) # 위 출력을 2D 텐서(samples, maxlen*8)로 펼침

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train
                    , epochs=10
                    , batch_size=32
                    , validation_split=0.2)

# 약 75%의 검증 정확도 확보
# but, 임베딩 시퀀스를 펼치고 하나의 Dense층을 훈련했으므로 입력 시퀀스에 있는 각 단어를 독립적으로 다룸
# 즉, 단어 사이 관계나 문장구조를 고려하지 않음
# 따라서 각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환층이나 1D 합성곱 층을 추가하는 것이 좋음

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 3s 129us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 1s 68us/step - loss: 0.5657 - acc: 0.

#### 사전 훈련된 단어 임베딩 사용하기

- 미리 계산된 임베딩 공간에서 임베딩 백터를 로드
- 위 공간은 언어 구조의 일반적인 측면을 잡아낼 수 있음
- 충분한 데이터가 없어서 자신만의 좋은 특성을 학습하지 못하지만 꽤 일반적인 특성이 필요할 때 사용 like 위의 20개 단어 케이스
---
- **Word2vec 알고리즘**: 가장 성공적인 단어 임베딩 방법
  - Word2vec의 차원은 <u>성별처럼 구체적인 의미가 있는 속성</u>을 잡아냄

- 케라스의 Embedding층을 위해 내려받을 수 있는 미리 계산된 단어 임베딩 데이터베이스  
  1) Word2vec  
  2) GloVe (by Stanford Univ., 2014)
    - 단어의 동시 출현 통계를 기록한 행렬을 분해하는 기법 사용

## 6.1.3 모든 내용을 적용하기: 원본 텍스트에서 단어 임베딩까지

In [9]:
# 원본 IMDB 텍스트 내려받기
# 훈련용 리뷰 하나를 문자열 하나로 만들어 훈련 데이터를 문자열의 리스트로 구성 후 긍/부정 labeling까지.
import os
imdb_dir = './datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg','pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [21]:

print(texts[23000])
print(labels[23000])

Clint Eastwood has definitely produced better movies than this, but this one does not embarrass him. Dirty Harry catches everyone's attention and unless one wants to watch romance, there is no reason why you won't like him. He is cool because he is dirty, is great because he kills without much thinking, is perfect because he gets the bullet right through your heart and a hero because he doesn't care.<br /><br />From what I have seen in movies in which Eastwood acts, the character of the lead role always captivates the audience. In White Hunter Black heart, he is the crazy director, in "in the Line of Fire" he is the "Old 'un" while here is the "almost" jobless with his job, that is to say he makes work for himself, doesn't care one damn about his superiors who practically send him out for a vacation.<br /><br />Based on a rape victim, this movie is promising for all the "no non-sense" movie watchers. The movie has nothing that goes away from he central plot. However, what makes it slig

#### 데이터 토큰화

In [24]:
# 텍스트 벡터로 만들고 훈련세트와 검증세트 나누기
# 사전 훈련된 단어 임베딩은 훈련 데이터가 부족할 때 특히 유용하므로 해당 상황 연출(훈련 데이터를 처음 200개 샘플로 제한)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100 # 100개 이후 단어는 버림
training_samples = 200 # 훈련 샘플은 200개
val_samples = 10000
max_words = 10000 # 데이터셋에서 빈도 높은 1만개 단어만 사용

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

88582개의 고유한 토큰을 찾았습니다.


In [25]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('데이터 텐서의 크기: ', data.shape)
print('레이블 텐서의 크기: ', labels.shape)

데이터 텐서의 크기:  (25000, 100)
레이블 텐서의 크기:  (25000,)


In [26]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + val_samples]
y_val = labels[training_samples: training_samples + val_samples]

In [31]:
x_train.shape # wjdtn 100개씩 200 세트

(200, 100)

#### GloVe 단어 임베딩 내려받기